In [1]:
!pip install torch transformers flask beautifulsoup4 requests nltk

In [2]:
pip install accelerate -U


  Using cached accelerate-0.29.3-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.1
    Uninstalling accelerate-0.29.1:
      Successfully uninstalled accelerate-0.29.1
  Rolling back uninstall of accelerate
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-config.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-0m3a57ju\accelerate-config.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-estimate-memory.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-0m3a57ju\accelerate-estimate-memory.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-launch.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-0m3a57ju\accelerate-launch.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate.exe
   from C:\Users\WinTo\AppData\Lo

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\accelerate.exe' -> 'c:\\Python311\\Scripts\\accelerate.exe.deleteme'



In [3]:
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import torch

In [4]:
# Tokenizer and model setup
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125M')
tokenizer.pad_token = tokenizer.eos_token

In [5]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, inputs, outputs, max_length=512):
        self.tokenizer = tokenizer
        self.inputs = inputs
        self.outputs = outputs
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]
        
        input_encodings = tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length)
        output_encodings = tokenizer(output_text, padding="max_length", truncation=True, max_length=self.max_length)
        
        return {
            "input_ids": torch.tensor(input_encodings['input_ids']),
            "attention_mask": torch.tensor(input_encodings['attention_mask']),
            "labels": torch.tensor(output_encodings['input_ids'])
        }
    

# Load and preprocess data
df = pd.read_json('Psychology-10K.json')
inputs = df['input'].tolist()
outputs = df['output'].tolist()
dataset = TextDataset(tokenizer, inputs, outputs)

In [11]:
# Training arguments and setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=7,
    per_device_eval_batch_size=5, 
    logging_steps=20,
    warmup_steps=500,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


In [12]:
# Train the model
trainer.train()

  0%|          | 0/2814 [00:00<?, ?it/s]

{'loss': 14.9627, 'grad_norm': 68875.265625, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 15.4287, 'grad_norm': 14919.169921875, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}
{'loss': 16.2641, 'grad_norm': 28632.544921875, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 14.1824, 'grad_norm': 186405.984375, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.06}
{'loss': 11.9143, 'grad_norm': 333991.78125, 'learning_rate': 1e-05, 'epoch': 0.07}
{'loss': 11.6098, 'grad_norm': 144124.84375, 'learning_rate': 1.2e-05, 'epoch': 0.09}
{'loss': 9.4844, 'grad_norm': 67671.4453125, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.1}
{'loss': 6.3367, 'grad_norm': 377318.15625, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.11}
{'loss': 5.2088, 'grad_norm': 469778.96875, 'learning_rate': 1.8e-05, 'epoch': 0.13}


KeyboardInterrupt: 

In [ ]:
def predict_response(input_text, model, tokenizer):
    # Preprocess and tokenize the input text
    encoded_input = tokenizer.encode_plus(
        input_text,
        max_length=512,  # Ensure the length matches what was used during training
        truncation=True,
        padding='max_length',
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Move the tensors to the same device as the model
    encoded_input = {key: tensor.to(model.device) for key, tensor in encoded_input.items()}

    # Make prediction
    with torch.no_grad():
        output = model(**encoded_input)

    # Retrieve the probabilities (logits) and find the predicted class
    logits = output.logits
    predicted_class_index = logits.argmax(-1).item()

    # If you used LabelEncoder during preprocessing, you need to reverse that encoding:
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label

# Example usage
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Make sure the model is on the right device
input_text = "i feel lonely"
response = predict_response(input_text, model, tokenizer)
print("Predicted Response:", response)

NameError: name 'model' is not defined